In [1]:
#%matplotlib notebook
import glob
#import io
import os
import cv2
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from svglib.svglib import svg2rlg
from collections import defaultdict
from fiona.crs import from_epsg
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import numpy as np

from archigan.datalayer import Layer
from archigan.pipeline import ArchiPipeline
from archigan.boston import ParcelInputLayer, ParcelOutputLayer, parse_GIS_bostonbuildings_2016
from archigan.cvc_fp import FootprintInputLayer, RepartitionInputLayer, RepartitionOutputLayer, parse_CVC_FP_svg

In [2]:
path = 'datasets/boston_buildings/2016'
x, y = 800, 1000
parsed = parse_GIS_bostonbuildings_2016(path)
parsed = {f'boston_{(j + x):04d}': sample for j, sample in enumerate(parsed[x:y])}

directory = './tmp/'

print('N:', len(parsed))

#ParcelInputLayer.samples_to_imgs(parsed, directory)
ParcelOutputLayer.samples_to_imgs(parsed, directory)

Reading parcel data: datasets/boston_buildings/2016/Parcels_2016_Data_Full.shp
Reading footprint data: datasets/boston_buildings/2016/boston_buildings.shp
Reprojecting footprint data to EPSG: 3857


/home/cogle/dev/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))



N: 200


/home/cogle/dev/anaconda3/lib/python3.7/site-packages/matplotlib/tight_layout.py:231: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [3]:
svgs = glob.glob('datasets/ImagesGT/*.svg')
classes = (
    'Door',
    'Window',
    'Room',
    'Wall',
    'Separation',
    'Parking',
)
parsed = {svg: parse_CVC_FP_svg(svg, classes) for svg in svgs}
directory = './tmp/'

FootprintInputLayer.samples_to_imgs(parsed, directory)
RepartitionInputLayer.samples_to_imgs(parsed, directory)
RepartitionOutputLayer.samples_to_imgs(parsed, directory)

In [5]:
stages = (
    'ParcelInputLayer',
    'ParcelOutputLayer',
    'FootprintInputLayer',
    'RepartitionInputLayer',
    'RepartitionOutputLayer',
)
layers = [os.path.join('./prepared/parcel2floorplan_5layer', layer) for layer in stages]
stages = [(j - 1, j) for j in range(1, len(layers))]
stages.pop(1)
directory = './prepared/parcel2floorplan_3stage'

print(layers, stages, directory)
pipeline = ArchiPipeline(layers, stages)

['./prepared/parcel2floorplan_5layer/ParcelInputLayer', './prepared/parcel2floorplan_5layer/ParcelOutputLayer', './prepared/parcel2floorplan_5layer/FootprintInputLayer', './prepared/parcel2floorplan_5layer/RepartitionInputLayer', './prepared/parcel2floorplan_5layer/RepartitionOutputLayer'] [(0, 1), (2, 3), (3, 4)] ./prepared/parcel2floorplan_3stage


In [6]:
pipeline.setup_training(directory)

108 samples in train: 100%|██████████| 122/122 [00:00<00:00, 239.64it/s]


In [7]:
import geopandas as gpd

df = gpd.read_file('datasets/boston_streets/Boston_Street_Segments.shp')
df.head()

,OBJECTID,SEGMENT_ID,L_F_ADD,L_T_ADD,R_F_ADD,R_T_ADD,PRE_DIR,ST_NAME,ST_TYPE,SUF_DIR,...,PLACE00_R,TRACT00_L,TRACT00_R,BLOCK00_L,BLOCK00_R,MCD00_L,MCD00_R,STREET_ID,SHAPElen,geometry
0,1,1.0,12,22,13,29,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,166.673187,"LINESTRING (776005.375 2949518.000, 776115.625..."
1,2,2.0,24,42,31,43,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,162.142997,"LINESTRING (776115.625 2949643.000, 776219.501..."
2,3,3.0,44,64,45,67,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,178.522006,"LINESTRING (776219.501 2949767.499, 776336.312..."
3,4,4.0,66,72,69,71,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,196.841018,"LINESTRING (776336.312 2949902.500, 776466.375..."
4,5,5.0,74,90,73,87,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,179.277949,"LINESTRING (776466.375 2950050.250, 776584.622..."
